# Random sample generation

In [1]:

from xpress import *
import numpy as np
np.random.seed(8)  # Set the seed for reproducibility

N_scenarios =  5 # Number of scenarios to generate
def scenario_generation(N_scenarios):
    KT_values = {}  # Dictionary to store the generated values of KT
    for scenario in range(1, N_scenarios+1):
        KT_values[scenario] = np.random.poisson(100)  # Generate a Poisson-distributed value with mean 100
    return KT_values

# Accessing the KT values for a specific scenario
scenario_number = 1  # Example scenario number
KT_values = scenario_generation(20)
KT_value = KT_values[scenario_number]
#print(f"KT value for scenario {scenario_number}: {KT_value}")
print(KT_values)

{1: 113, 2: 92, 3: 98, 4: 101, 5: 102, 6: 108, 7: 103, 8: 95, 9: 87, 10: 82, 11: 97, 12: 96, 13: 120, 14: 98, 15: 110, 16: 81, 17: 109, 18: 99, 19: 90, 20: 91}


# Items generation

In [2]:
import random

def generate_item_data(num_items):
    volumes = {}
    demand = {}
    costs = {}
    alpha = {}
    beta = {}
    price = {}
    display_min = {}

    for i in range(1, num_items + 1):
        volumes[i] = int(random.uniform(0.01, 10))  # Random volume for item i
        demand[i] = int(random.randint(1, 16))  # Random demand for item i
        costs[i] = int(random.uniform(1.0, 101.0))  # Random cost for item i
        price[i] = 2 * costs[i]
        display_min[i] = int(random.uniform(50,70))

        for j in range(1, num_items + 1):
            if i == j:
                alpha[(j, i)] = 0.0
                beta[(j, i)] = 0.0
            else:
                alpha[(j, i)] = float("{:.2f}".format(random.uniform(0.1, 0.9)))  # Random alpha for item j -> item i
                beta[(j, i)] = float("{:.2f}".format(random.uniform(0.1, 0.9)))  # Random beta for item j -> item i

    return volumes, demand, costs, alpha, beta, price, display_min


# Specify the number of items
num_items = 20

# Generate item data
volumes, demand, costs, alpha, beta,price, display_min = generate_item_data(num_items)


# Stochastic program

In [7]:
import xpress as xp
import numpy as np

#defining the problem
num_items = 50
volumes, demand, costs, alpha, beta,price, display_min = generate_item_data(num_items)
N_scenarios = 10
KT_values = scenario_generation(N_scenarios)



# Define the set of items
N = list(range(1, num_items + 1))  # Example set of items
lim_prod = 30
# Define the set of scenarios
Omega = list(range(1, len(KT_values) + 1))  # Example set of scenarios

w = volumes
d = demand
alpha = alpha
beta = beta
p = price
c = costs
display_min = display_min
C = 100 * np.dot(np.array(list(d.values())), np.array(list(w.values())))  # Example budget constraint
M = 1000000000000000000000000  # Example capacity constraint
delta = 0.7  # Example substitution constraint
KT = KT_values

# Define the model
model = xp.problem(name="Stochastic Assortment Optimization")


# Define the decision variables
x = np.array([xp.var(vartype=xp.binary, name=f'x_{i}') for i in N])
m = {(i,omega): xp.var(vartype=xp.integer, lb=0, name=f'm_{i}_{omega}')for omega in Omega for i in N}
u = np.array([xp.var(vartype=xp.integer, lb=0, name=f'u_{i}') for i in N])
mu = {(i,omega): xp.var(vartype=xp.binary, name=f'mu_{i}_{omega}')for omega in Omega for i in N}
mu_1 = {(i,omega): xp.var(vartype=xp.binary, name=f'mu_1_{i}_{omega}')for omega in Omega for i in N}
v_plus = {(i, omega): xp.var(vartype=xp.integer, lb=0, name=f'v_plus_{i}_{omega}') for omega in Omega for i in N}
v_minus = {(i, omega): xp.var(vartype=xp.integer, lb=0, name=f'v_minus_{i}_{omega}') for i in N for omega in Omega}
q_dict = {
    (i, j, omega): xp.var(vartype=xp.integer, lb=0, name=f'q_{i}_{j}_{omega}')
    for i in N for j in N for omega in Omega
}
s_dict = {
    (i, j, omega): xp.var(vartype=xp.integer, lb=0, name=f's_{i}_{j}_{omega}')
    for i in N for j in N  for omega in Omega
}
#model.addVariables(x, m, u, v_plus, v_minus, q_dict, s_dict, mu, mu_1)
model.addVariable(x)
model.addVariable(m)
model.addVariable(u)
model.addVariable(v_plus)
model.addVariable(v_minus)
model.addVariable(q_dict)
model.addVariable(s_dict)
model.addVariable(mu)
model.addVariable(mu_1)


# Define the objective function
expected_profit = xp.Sum(
    xp.Sum(m[(i,omega)] * p[i]  for i in N) for omega in Omega
) / len(Omega)

assortment_substitution_profit = xp.Sum(xp.Sum(xp.Sum(
    p[i] * delta * alpha[(j, i)] * q_dict[(j,i,omega)] for i in N) for j in N )
     for omega in Omega) / len(Omega)
stockout_substitution_profit = xp.Sum(xp.Sum(xp.Sum(
    p[i] * delta * beta[(j, i)] * s_dict[(j,i,omega)] for i in N) for j in N)
     for omega in Omega) / len(Omega)
cost_of_storage = xp.Sum(c[i] * u[i-1] for i in N)

#obj_constraint = xp.Sum(xp.Sum(10000000*(KT[omega]*d[j]-m[(j,omega)]-xp.Sum(delta * beta[(j,i)] * s_dict[(j,i,omega)] for i in N)+KT[omega] * d[j]-xp.Sum(delta * alpha[(j,i)]* q_dict[(j,i,omega)] for i in N)) for j in N)for omega in Omega)


model.setObjective(
    expected_profit + stockout_substitution_profit + assortment_substitution_profit - cost_of_storage,
    sense=xp.maximize,
)

# Define the constraints
budget_constraint = xp.Sum(w[i] * u[i-1] for i in N) <= C
display_constraint = [u[i-1] >= display_min[i]*x[i-1] for i in N]
nbr_prods_constraint = xp.Sum(x[i-1] for i in N)<=lim_prod

capacity_constraints = [u[i-1] <= M * x[i-1] for i in N]  # Changed to a list comprehension
nullity_constraints = [x[i-1]<=u[i-1] for i in N]

min_constraints = []
for i in N:
    for omega in Omega:
        min_constraints.append(m[(i,omega)] == xp.min(u[i-1],KT[omega]*d[i]))
        #min_constraints.append(m[(i,omega)] <= KT[omega]*d[i])
        #min_constraints.append(m[(i,omega)] <= u[i-1])


assortment_constraints = []
for omega in Omega:
    for i in N:
        for j in N:
            if j == i:
                q_dict[j,i,omega] = 0
            if j != i:
                assortment_constraints.append(q_dict[j,i,omega] <= M*x[i-1])
                assortment_constraints.append(q_dict[j,i,omega] <= M*(1-x[j-1]))
        '''assortment_constraints.append(xp.Sum(q_dict[(j,i,omega)] for j in N if j!= i) <= u[i-1] - m[(i,omega)])
    for j in N:
        assortment_constraints.append(xp.Sum(delta * alpha[(j,i)]*q_dict[(j,i,omega)] for i in N if i != j) <= KT[omega]*d[j])'''

        
lost_sales_constraints = []

for omega in Omega:
    for i in N:
        lost_sales_constraints.append(v_plus[(i,omega)]-v_minus[(i,omega)] == m[(i,omega)] +xp.Sum(q_dict[(j,i,omega)] for j in N if j != i) -u[i-1])
        lost_sales_constraints.append(v_plus[(i,omega)] <= M * mu[(i,omega)])
        lost_sales_constraints.append(v_minus[(i,omega)] <= M * mu_1[(i,omega)])
        lost_sales_constraints.append(mu[(i,omega)] + mu_1[(i,omega)] == 1)

stock_out_constraints = []
for i in N:
    for j in N:
        for omega in Omega:
            if i == j:
                s_dict[i,j,omega] = 0
            if j != i:
                stock_out_constraints.append(s_dict[(j, i, omega)] <= M * x[i-1])
                stock_out_constraints.append(s_dict[(j, i, omega)] <= M * x[j-1])
                
'''for omega in Omega:
    for i in N:
        stock_out_constraints.append(xp.Sum(delta * beta[(j,i)] * s_dict[(j,i,omega)] for j in N if j != i) <= v_minus[(i,omega)])
    for j in N:
        stock_out_constraints.append(xp.Sum(delta * beta[(j,i)] *  s_dict[(j,i,omega)] for i in N if i != j) <= KT[omega] * d[j] - m[(j,omega)])'''
global_constraints = []
for omega in Omega:
    for i in N:
        global_constraints.append(xp.Sum(m[(i,omega)] + xp.Sum(q_dict[(i,j,omega)] for j in N if j != i)
                                         + xp.Sum(s_dict[(i,j,omega)] for j in N if j != i)) == KT[omega] * d[i])
        
global_capa_constraints = []
for omega in Omega:
    for i in N:
        global_capa_constraints.append(xp.Sum(m[(i,omega)] + xp.Sum(q_dict[(j,i,omega)] for j in N if j != i)
                                             + xp.Sum(s_dict[(j,i,omega)] for j in N if j != i)) <= u[i-1])
        


model.addConstraint(budget_constraint)
model.addConstraint(capacity_constraints)  
model.addConstraint(nullity_constraints)
model.addConstraint(assortment_constraints)
model.addConstraint(min_constraints)
model.addConstraint(lost_sales_constraints)
model.addConstraint(stock_out_constraints)
model.addConstraint(nbr_prods_constraint)
model.addConstraint(display_constraint)
model.addConstraint(global_constraints)
model.addConstraint(global_capa_constraints)


# Solve the problem

model.solve()
# Get the optimal objective value
optimal_objective_value = model.getObjVal()
print("optimal objective",optimal_objective_value)

# Get the optimal decision variable values
optimal_x = {i: model.getSolution(x[i-1]) for i in N}
optimal_u = {i: model.getSolution(u[i-1]) for i in N}
optimal_s = {(i, j, omega): model.getSolution(s_dict[(i, j, omega)]) for i in N for j in N if j != i for omega in Omega}
optimal_q = {(i, j, omega): model.getSolution(q_dict[(i, j, omega)]) for i in N for j in N if j != i for omega in Omega}
optimal_m = {(i,omega): model.getSolution(m[(i, omega)]) for i in N for omega in Omega}
optimal_v_plus = {(i,omega):model.getSolution(v_plus[(i,omega)]) for i in N for omega in Omega}
optimal_v_minus = {(i,omega):model.getSolution(v_minus[(i,omega)]) for i in N for omega in Omega}
optimal_mu = {(i,omega):model.getSolution(mu[(i,omega)]) for i in N for omega in Omega}
optimal_mu_1 = {(i,omega):model.getSolution(mu_1[(i,omega)]) for i in N for omega in Omega}

Original problem size
   linear:    101652 rows, 52601 columns, 325896 linear coefficients
   nonlinear: 500 coefficients, 3000 tokens
Nonlinear presolve
   checking equivalence to general constraints ABS/MIN/MAX and PWL reformulations
   converted 400 MIN/MAX to optimizer MIP constructs
   converted 500 formulas to linear constraints
   problem is equivalent, reformulated as a MIP problem
Presolved problem size
   linear:    101652 rows, 53001 columns, 326396 linear coefficients
Problem is nonlinear presolved
Maximizing problem using Xpress-Optimizer
FICO Xpress v9.0.0, Hyper, solve started 19:42:57, Jul 14, 2023
Heap usage: 57MB (peak 58MB, 28MB system)
Maximizing MILP Stochastic Assortment Optimization using up to 8 threads and up to 63GB memory, with these control settings:
OUTPUTLOG = 1
IFCHECKCONVEXITY = 0
Original problem has:
    101652 rows        53001 cols       326396 elements     52600 entities
       400 gencons
Presolved problem has:
     76316 rows        50720 cols    

# Display non negative results

In [14]:
non_zero_s = {key: value for key, value in optimal_s.items() if value != 0.0}
non_zero_q = {key: value for key, value in optimal_q.items() if value != 0.0}
non_zero_u = {key: value for key, value in optimal_u.items() if value != 0.0}
non_zero_m = {key: value for key, value in optimal_m.items() if value != 0.0}
non_zero_x = {key: value for key, value in optimal_x.items() if value != 0.0}
print("s_dict",non_zero_s)
print("q_dict",non_zero_q)
print("u_val",non_zero_u)
print("m_val",non_zero_m)
print("x_val",non_zero_x)

s_dict {(8, 35, 8): 31.999999999999545, (12, 9, 8): 22.0, (22, 5, 8): 21.999999999999897, (23, 35, 8): 21.999999999999545, (41, 35, 8): 32.00000000000023, (42, 40, 8): 28.0}
q_dict {(1, 27, 1): 134.9999999999999, (1, 27, 2): 180.0, (1, 27, 3): 90.00000000000011, (1, 27, 4): 170.9999999999998, (1, 27, 5): 17.999999999999886, (1, 27, 6): 63.00000000000034, (1, 27, 7): 17.999999999999886, (1, 27, 9): 170.9999999999999, (1, 27, 10): 117.00000000000011, (1, 49, 1): 61.000000000000114, (1, 49, 3): 115.99999999999989, (1, 49, 4): 17.0000000000002, (1, 49, 5): 204.0000000000001, (1, 49, 6): 148.99999999999966, (1, 49, 7): 204.0000000000001, (1, 49, 8): 226.0, (1, 49, 9): 17.000000000000114, (1, 49, 10): 82.99999999999989, (2, 23, 1): 120.00000000000045, (2, 23, 3): 88.00000000000045, (2, 23, 6): 55.000000000000455, (2, 23, 10): 121.00000000000045, (2, 27, 2): 27.000000000001705, (2, 35, 3): 164.99999999999955, (2, 35, 5): 333.0, (2, 35, 6): 228.0000000075449, (2, 35, 7): 333.0, (2, 35, 8): 339

# Printing demand

In [24]:
print([((j,omega), KT[omega]*d[j]) for j in N for omega in Omega])

[((1, 1), 500), ((1, 2), 550), ((2, 1), 100), ((2, 2), 110), ((3, 1), 800), ((3, 2), 880), ((4, 1), 800), ((4, 2), 880), ((5, 1), 500), ((5, 2), 550)]


In [18]:
optimal_u

{1: 565.0, 2: 1808.0, 3: 1017.0, 4: 904.0, 5: 678.0}

# Measuring runtime

In [25]:
import xpress as xp
import numpy as np
import time

def measure_runtime(N_scenarios, num_items):
    # defining the problem
    volumes, demand, costs, alpha, beta, price, display_min = generate_item_data(num_items)
    KT_values = scenario_generation(N_scenarios)

    # Define the set of items
    N = list(range(1, num_items + 1))  # Example set of items
    lim_prod = 7
    # Define the set of scenarios
    Omega = list(range(1, len(KT_values) + 1))  # Example set of scenarios

    w = volumes
    d = demand
    alpha = alpha
    beta = beta
    p = price
    c = costs
    display_min = display_min
    C = 100 * np.dot(np.array(list(d.values())), np.array(list(w.values())))  # Example budget constraint
    M = 1000000000000000000000000  # Example capacity constraint
    delta = 0.7  # Example substitution constraint
    KT = KT_values

    # Define the model
    model = xp.problem(name="Stochastic Assortment Optimization")

    # Define the decision variables
    x = np.array([xp.var(vartype=xp.binary, name=f'x_{i}') for i in N])
    m = {(i, omega): xp.var(vartype=xp.integer, lb=0, name=f'm_{i}_{omega}') for omega in Omega for i in N}
    u = np.array([xp.var(vartype=xp.integer, lb=0, name=f'u_{i}') for i in N])
    mu = {(i, omega): xp.var(vartype=xp.binary, name=f'mu_{i}_{omega}') for omega in Omega for i in N}
    mu_1 = {(i, omega): xp.var(vartype=xp.binary, name=f'mu_1_{i}_{omega}') for omega in Omega for i in N}
    v_plus = {(i, omega): xp.var(vartype=xp.integer, lb=0, name=f'v_plus_{i}_{omega}') for omega in Omega for i in N}
    v_minus = {(i, omega): xp.var(vartype=xp.integer, lb=0, name=f'v_minus_{i}_{omega}') for i in N for omega in Omega}
    q_dict = {(i, j, omega): xp.var(vartype=xp.integer, lb=0, name=f'q_{i}_{j}_{omega}') for i in N for j in N for omega in Omega}
    s_dict = {(i, j, omega): xp.var(vartype=xp.integer, lb=0, name=f's_{i}_{j}_{omega}') for i in N for j in N for omega in Omega}

    model.addVariable(x)
    model.addVariable(m)
    model.addVariable(u)
    model.addVariable(v_plus)
    model.addVariable(v_minus)
    model.addVariable(q_dict)
    model.addVariable(s_dict)
    model.addVariable(mu)
    model.addVariable(mu_1)

    # Define the objective function
    expected_profit = xp.Sum(xp.Sum(m[(i, omega)] * p[i] for i in N) for omega in Omega) / len(Omega)
    assortment_substitution_profit = xp.Sum(
        xp.Sum(xp.Sum(p[i] * delta * alpha[(j, i)] * q_dict[(j, i, omega)] for i in N) for j in N) for omega in Omega
    ) / len(Omega)
    stockout_substitution_profit = xp.Sum(
        xp.Sum(xp.Sum(p[i] * delta * beta[(j, i)] * s_dict[(j, i, omega)] for i in N) for j in N) for omega in Omega
    ) / len(Omega)
    cost_of_storage = xp.Sum(c[i] * u[i - 1] for i in N)

    model.setObjective(
        expected_profit + stockout_substitution_profit + assortment_substitution_profit - cost_of_storage,
        sense=xp.maximize,
    )

    # Define the constraints
    budget_constraint = xp.Sum(w[i] * u[i - 1] for i in N) <= C
    display_constraint = [u[i - 1] >= display_min[i] * x[i - 1] for i in N]
    nbr_prods_constraint = xp.Sum(x[i - 1] for i in N) <= lim_prod

    capacity_constraints = [u[i - 1] <= M * x[i - 1] for i in N]  # Changed to a list comprehension
    nullity_constraints = [x[i - 1] <= u[i - 1] for i in N]

    min_constraints = []
    for i in N:
        for omega in Omega:
            min_constraints.append(m[(i, omega)] == xp.min(u[i - 1], KT[omega] * d[i]))

    assortment_constraints = []
    for omega in Omega:
        for i in N:
            for j in N:
                if j == i:
                    q_dict[j, i, omega] = 0
                if j != i:
                    assortment_constraints.append(q_dict[j, i, omega] <= M * x[i - 1])
                    assortment_constraints.append(q_dict[j, i, omega] <= M * (1 - x[j - 1]))

    lost_sales_constraints = []
    for omega in Omega:
        for i in N:
            lost_sales_constraints.append(
                v_plus[(i, omega)] - v_minus[(i, omega)] == m[(i, omega)] + xp.Sum(
                    q_dict[(j, i, omega)] for j in N if j != i
                ) - u[i - 1]
            )
            lost_sales_constraints.append(v_plus[(i, omega)] <= M * mu[(i, omega)])
            lost_sales_constraints.append(v_minus[(i, omega)] <= M * mu_1[(i, omega)])
            lost_sales_constraints.append(mu[(i, omega)] + mu_1[(i, omega)] == 1)

    stock_out_constraints = []
    for i in N:
        for j in N:
            for omega in Omega:
                if i == j:
                    s_dict[i, j, omega] = 0
                if j != i:
                    stock_out_constraints.append(s_dict[(j, i, omega)] <= M * x[i - 1])
                    stock_out_constraints.append(s_dict[(j, i, omega)] <= M * x[j - 1])

    global_constraints = []
    for omega in Omega:
        for i in N:
            global_constraints.append(
                xp.Sum(
                    m[(i, omega)] + xp.Sum(q_dict[(i, j, omega)] for j in N if j != i) + xp.Sum(
                        s_dict[(i, j, omega)] for j in N if j != i
                    )
                ) == KT[omega] * d[i]
            )

    global_capa_constraints = []
    for omega in Omega:
        for i in N:
            global_capa_constraints.append(
                xp.Sum(
                    m[(i, omega)] + xp.Sum(q_dict[(j, i, omega)] for j in N if j != i) + xp.Sum(
                        s_dict[(j, i, omega)] for j in N if j != i
                    )
                ) <= u[i - 1]
            )

    model.addConstraint(budget_constraint)
    model.addConstraint(capacity_constraints)
    model.addConstraint(nullity_constraints)
    model.addConstraint(assortment_constraints)
    model.addConstraint(min_constraints)
    model.addConstraint(lost_sales_constraints)
    model.addConstraint(stock_out_constraints)
    model.addConstraint(nbr_prods_constraint)
    model.addConstraint(display_constraint)
    model.addConstraint(global_constraints)
    model.addConstraint(global_capa_constraints)

    # Solve the problem
    start_time = time.time()
    model.solve()
    end_time = time.time()
    runtime = end_time - start_time

    # Get the optimal objective value
    optimal_objective_value = model.getObjVal()
    print("Optimal objective:", optimal_objective_value)

    # Get the optimal decision variable values
    optimal_x = {i: model.getSolution(x[i - 1]) for i in N}
    optimal_u = {i: model.getSolution(u[i - 1]) for i in N}
    optimal_s = {(i, j, omega): model.getSolution(s_dict[(i, j, omega)]) for i in N for j in N if j != i for omega in Omega}
    optimal_q = {(i, j, omega): model.getSolution(q_dict[(i, j, omega)]) for i in N for j in N if j != i for omega in Omega}
    optimal_m = {(i, omega): model.getSolution(m[(i, omega)]) for i in N for omega in Omega}
    optimal_v_plus = {(i, omega): model.getSolution(v_plus[(i, omega)]) for i in N for omega in Omega}
    optimal_v_minus = {(i, omega): model.getSolution(v_minus[(i, omega)]) for i in N for omega in Omega}
    optimal_mu = {(i, omega): model.getSolution(mu[(i, omega)]) for i in N for omega in Omega}
    optimal_mu_1 = {(i, omega): model.getSolution(mu_1[(i, omega)]) for i in N for omega in Omega}

    return runtime


In [28]:
runtimes = []
input_sample = []

In [35]:
updatable_input = []
N_scen = 60
N_prod = 60
for i in range (45,N_scen + 1,5):
    for j in range(45,N_prod + 1,5):
        input_sample.append((i,j))
        updatable_input.append((i,j))

32 16


In [36]:
updatable_runtimes = []
for ele in updatable_input:
    a = measure_runtime(ele[0],ele[1])
    runtimes.append(a)
    updatable_runtimes.append(a)
    

Original problem size
   linear:    370712 rows, 192466 columns, 1187007 linear coefficients
   nonlinear: 2025 coefficients, 12150 tokens
Nonlinear presolve
   checking equivalence to general constraints ABS/MIN/MAX and PWL reformulations
   converted 1080 MIN/MAX to optimizer MIP constructs
   converted 2025 formulas to linear constraints
   problem is equivalent, reformulated as a MIP problem
Presolved problem size
   linear:    370712 rows, 193546 columns, 1189032 linear coefficients
Problem is nonlinear presolved
Maximizing problem using Xpress-Optimizer
FICO Xpress v9.0.0, Hyper, solve started 11:30:34, Jul 15, 2023
Heap usage: 205MB (peak 210MB, 114MB system)
Maximizing MILP Stochastic Assortment Optimization using up to 8 threads and up to 63GB memory, with these control settings:
OUTPUTLOG = 1
IFCHECKCONVEXITY = 0
Original problem has:
    370712 rows       193546 cols      1189032 elements    192465 entities
      1080 gencons
Presolved problem has:
    277259 rows       1832